In [17]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, types
import datetime
import requests
import time

In [2]:
username = 'TALENT_LMS_KEY'
password = ''


mysql_host = 'HOSTNAME'
mysql_root = 'USERNAME'
mysql_password = 'PASSWORD'
mysql_db = 'DATABASE_NAME'


mysql_engine = create_engine(f'mysql+pymysql://{mysql_root}:{mysql_password}@{mysql_host}:3306/{mysql_db}')

In [15]:
def load_lms_courses():
    retries = 1
    success = False
    
    while not success:
        try:
            url = 'https://thirdspace.talentlms.com/api/v1/courses'
            response = requests.get(url, auth=(username, password), timeout=5)
            course_data = response.json()
            success = True

        except requests.exceptions.ConnectionError:
            wait = retries * 5
            print(f'Error! Waiting {wait} secs and re-trying...')
            time.sleep(wait)
            retries += 1
    
    courses_df = pd.json_normalize(course_data)

    courses_df['course_created_at'] =  courses_df['creation_date'].\
            apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y, %H:%M:%S'))

    courses_df['updated_at'] =  courses_df['last_update_on'].\
            apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y, %H:%M:%S'))
    
    COLS = ['id', 'name', 'category_id', 'description', 'status', 
            'creator_id', 'hide_from_catalog', 'time_limit', 'level',
            'shared', 'course_created_at', 'updated_at']

    lms_courses = courses_df[COLS]

    return lms_courses


def get_max_updated_at(table_name):
    conn = mysql_engine.connect()
    result = conn.execute(f"select coalesce(max(updated_at),'1970-01-01 00:00:01') from {mysql_db}.{table_name}").fetchall()
    max_updated_at = result[0][0]
    return max_updated_at


def get_latest_df(df, table_name):
    max_timestamp = get_max_updated_at(table_name)
    temp_df = df[df['updated_at'] >= max_timestamp]
    return temp_df


def create_temp_courses_table(df):
    
    sql_types = {
            'id' : types.INTEGER(),
            'name': types.VARCHAR(length=255),
            'category_id': types.INTEGER(),
            'description' : types.TEXT(),
            'status': types.VARCHAR(length=64),
            'creator_id': types.INTEGER(),
            'hide_from_catalog' : types.INTEGER(),
            'time_limit': types.INTEGER(),
            'level': types.INTEGER(),
            'shared' : types.INTEGER(),
            'course_created_at': types.TIMESTAMP(),
            'updated_at': types.TIMESTAMP()   
        }
        
    df.to_sql(
            name='temp_talent_lms_courses',
            con=mysql_engine,
            if_exists='replace',
            dtype=sql_types,
            index=False
        )


def upsert_courses_table():
    conn = mysql_engine.connect()
    
    sql_query = f'''

    insert into {mysql_db}.talent_lms_courses  
    select * from {mysql_db}.temp_talent_lms_courses 
    on duplicate key update 
        talent_lms_courses.`name`            = temp_talent_lms_courses.`name`,
        talent_lms_courses.category_id       = temp_talent_lms_courses.category_id,
        talent_lms_courses.`description`     = temp_talent_lms_courses.`description`,
        talent_lms_courses.`status`          = temp_talent_lms_courses.`status`,
        talent_lms_courses.creator_id        = temp_talent_lms_courses.creator_id,
        talent_lms_courses.hide_from_catalog = temp_talent_lms_courses.hide_from_catalog,
        talent_lms_courses.time_limit        = temp_talent_lms_courses.time_limit,
        talent_lms_courses.`level`           = temp_talent_lms_courses.`level`,
        talent_lms_courses.shared            = temp_talent_lms_courses.shared,
        talent_lms_courses.updated_at        = temp_talent_lms_courses.updated_at;
        
'''

    conn.execute(sql_query)

In [4]:
# Note: Want to create the table first for courses in the db

courses_df = load_lms_courses() # Extracting LMS data
temp_df = get_latest_df(df=courses_df, table_name='talent_lms_courses') # Temp table to update
create_temp_courses_table(temp_df) # Creates the temp table in MySQL
upsert_courses_table() # Upsert Operations